In [4]:
from google.colab import drive
drive.mount('/content/drive')
!pip install deepchem rdkit matplotlib networkx

from joblib import parallel_backend
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from deepchem.feat import MolGraphConvFeaturizer
from deepchem.feat import CircularFingerprint
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from deepchem.feat import ConvMolFeaturizer
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
import deepchem as dc
#from drive.MyDrive.code.methods import *
from drive.MyDrive.code.methods_fingerprints import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [5]:
smiles_tr,smiles_test, logSweeter_tr, logSweeter_test, families_tr, families_test = load_data("/content/drive/MyDrive/code/SweetnersDB.xlsx")

Index(['Name', 'logSw', 'Chemical family', 'Dataset', 'Smiles'], dtype='object')


In [6]:
X_tr,X_test,scaler_X = prepare_fingerprint(smiles_tr,smiles_test)
y_tr,y_test,scaler_y = prepare_targets(logSweeter_tr,logSweeter_test)

[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerator
[13:31:16] DEPRECATION WARNING: please use MorganGenerat

In [7]:
onehot_tr,onehot_test = prepare_famille_chimique(families_tr, families_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
# Combiner les fingerprints et l'encodage one-hot
representation_tr = np.hstack((X_tr, onehot_tr))
representation_test = np.hstack((X_test,onehot_test))

# Scoring régression logistique

In [9]:
sweet_thr = 0
ytr = pd.DataFrame(y_tr,columns=['label'])
target_tr =  ytr['label'].apply(lambda x: 0 if x <= sweet_thr else 1)
ytest = pd.DataFrame(y_test,columns=['label'])
target_test =  ytest['label'].apply(lambda x: 0 if x <= sweet_thr else 1)

model = LogisticRegression(max_iter=10000, random_state=42)
param_grid = {
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear', 'saga'],
        'l1_ratio': [0, 0.5, 1]  # Utilisé uniquement si penalty est 'elasticnet'
    }



In [10]:
# Utiliser un context manager pour définir le backend

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=1, verbose=2) ## n_jobs = -1?
grid_search.fit(representation_tr, target_tr)


Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_

[CV] END ........C=0.01, l1_ratio=0, penalty=l1, solver=saga; total time=   0.1s
[CV] END ........C=0.01, l1_ratio=0, penalty=l1, solver=saga; total time=   0.0s
[CV] END ........C=0.01, l1_ratio=0, penalty=l1, solver=saga; total time=   0.0s
[CV] END ........C=0.01, l1_ratio=0, penalty=l1, solver=saga; total time=   0.0s
[CV] END ........C=0.01, l1_ratio=0, penalty=l1, solver=saga; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV] END .......C=0.01, l1_ratio=0, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .......C=0.01, l1_ratio=0, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .......C=0.01, l1_ratio=0, penalty=l2, solver=lbfgs; total time=   0.0s


...

[CV] END .......C=100, l1_ratio=1, penalty=none, solver=saga; total time=  13.0s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END .......C=100, l1_ratio=1, penalty=none, solver=saga; total time=  21.2s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
300 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1169, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=10000, random_state=42),
             n_jobs=1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'l1_ratio': [0, 0.5, 1],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             verbose=2)

In [11]:
# Affichage des meilleurs paramètres et du meilleur score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-validation Score: {grid_search.best_score_}")

# Prédiction sur l'ensemble de test avec le meilleur modèle
best_model = grid_search.best_estimator_
y_pred = best_model.predict(representation_test)

Best Parameters: {'C': 0.1, 'l1_ratio': 0, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Cross-validation Score: 0.7892549019607843


In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Évaluation du modèle
accuracy = accuracy_score(target_test, y_pred)
conf_matrix = confusion_matrix(target_test, y_pred)
class_report = classification_report(target_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.8125
Confusion Matrix:
[[15  6]
 [ 6 37]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.71      0.71        21
           1       0.86      0.86      0.86        43

    accuracy                           0.81        64
   macro avg       0.79      0.79      0.79        64
weighted avg       0.81      0.81      0.81        64



# Scoring arbre de décision

In [ ]:
model = DecisionTreeClassifier(random_state=42)
param_grid = {
            'criterion': ['gini', 'entropy'],  # 'log_loss' peut être utilisé pour des problèmes de classification multiclasse
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['auto', 'sqrt', 'log2']
        }
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=1, verbose=2)

grid_search.fit(representation_tr, target_tr)

In [ ]:
# Affichage des meilleurs paramètres et du meilleur score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-validation Score: {grid_search.best_score_}")

# Prédiction sur l'ensemble de test avec le meilleur modèle
best_model = grid_search.best_estimator_
y_pred = best_model.predict(representation_test)



# Évaluation du modèle
accuracy = accuracy_score(target_test, y_pred)
conf_matrix = confusion_matrix(target_test, y_pred)
class_report = classification_report(target_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

# scoring random forest

In [ ]:
model = RandomForestClassifier(random_state=42)
param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['auto', 'sqrt', 'log2']
        }
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=1, verbose=2)

grid_search.fit(representation_tr, target_tr)

In [ ]:
 # Affichage des meilleurs paramètres et du meilleur score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-validation Score: {grid_search.best_score_}")

# Prédiction sur l'ensemble de test avec le meilleur modèle
best_model = grid_search.best_estimator_
y_pred = best_model.predict(representation_test)



# Évaluation du modèle
accuracy = accuracy_score(target_test, y_pred)
conf_matrix = confusion_matrix(target_test, y_pred)
class_report = classification_report(target_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)